[View in Colaboratory](https://colab.research.google.com/github/rishabh-vij/Fast.ai_MOOC/blob/master/Week%206/ANN%20and%20RNN%20from%20scratch.ipynb)

##Language model(ANN and RNN) from scracth

In [0]:
#!pip install opencv-python
#!apt update && apt install -y libsm6 libxext6
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
! pip3 install fastai
#!pip3 install torchvision
#! pip install torchtext

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

**We're going to download the collected works of Nietzsche to use as our data for this class.**

In [0]:
PATH='data/nietzsche/'


In [4]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

nietzsche.txt: 606kB [00:01, 417kB/s]                           

corpus length: 600893


In [9]:
text[:500]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--IF,\nindeed, it s'

In [0]:
chars=sorted(list(set(text)))
vocab_size=len(chars)+1
chars.insert(0,'\0')

In [12]:
' '.join(chars)[:-50]

'\x00 \n   ! " \' ( ) , - . 0 1 2 3 4 5 6 7 8 9 : ; = ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ ] _ a b c d e f'

**Map chars to indices and vice-versa**

In [0]:
char_indices={c:i for i,c in enumerate(chars)}
indices_char={i:c for i,c in enumerate(chars)}

**idx will be the data we use from now on - it simply converts all the characters to their index (based on the mapping above)**



In [14]:
idx=[char_indices[i] for i in text]
idx[:6]

[40, 42, 29, 30, 25, 27]

##3 character model

**We'll now try building a model from scratch using pytorch that given the first 3 characters predicts the 4th one**

In [0]:
cs=3 #organizing our data 
c1_dat = [idx[i]   for i in range(0, len(idx)-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs, cs)]

In [0]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)
#^ Inputs
y=np.stack(c4_dat)
#^ Output

In [17]:
# first 4 inputs
x1[:4], x2[:4], x3[:4]


(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [18]:
# first 4 outputs
y[:4]

array([30, 29,  1, 40])

## Creating and training model


In [0]:
n_hidden=256 # no. of hidden layers in our model
n_fac=42 # Size of the embedding matrix

In [0]:
class char3model(nn.Module):
  def __init__(self,vocab_size,n_fac):
    super().__init__()
    self.e=nn.Embedding(vocab_size,n_fac) # create embedding matrix
    
    self.l_lin = nn.Linear(n_fac,n_hidden) # layer that connects inputs layer and hidden layer
    self.l_hidden = nn.Linear(n_hidden,n_hidden) # layer that connects 2 hidden layers
    self.l_out = nn.Linear(n_hidden,vocab_size) # layer that connects hidden layer to output layer
    
  def forward(self,c1,c2,c3):
    in1= F.relu(self.l_lin(self.e(c1)))#passing each character input through embedding layer, linear layer and applying RELU
    in2= F.relu(self.l_lin(self.e(c2)))
    in3= F.relu(self.l_lin(self.e(c3)))
    
    h=V(torch.zeros(in1.size()).cuda()) #creating the hidden layer and putting it on GPU
    h= F.tanh(self.l_hidden(h+in1))
    h= F.tanh(self.l_hidden(h+in2))
    h= F.tanh(self.l_hidden(h+in3))
    return F.log_softmax(self.l_out(h))
  

In [0]:
md= ColumnarModelData.from_arrays('.',[-1],np.stack([x1,x2,x3], axis=1), y, bs=512) # We use fastai's ColumnarModelData to pass our data i.e. x1,x2,x3 to our forward method

In [0]:
model=char3model(vocab_size,n_fac).cuda() #creating a model from just created class

In [0]:
*xs,yt=next(iter(md.trn_dl))
t = model(*V(xs))


In [0]:
opt=optim.Adam(model.parameters(),1e-2)

In [30]:
fit(model,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.105959   1.10178   



[array([1.10178])]

In [0]:
set_lrs(opt,1e-3) #Learning rate annealing

In [32]:
fit(model,md,1,opt,F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.838118   0.587075  



[array([0.58707])]

##Testing ur model

In [0]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = model(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [41]:
get_next('you')

'n'

In [37]:
get_next('p. ')

'I'

In [40]:
get_next('lov')

'e'

In [39]:
get_next(' th')

'e'

## Recurrent Neurel net(RNN) from scratch
**Now we'll create a model that given 8 characters can predict the 9th character**

In [0]:
cs=8
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]


In [0]:
xs = np.stack(c_in_dat, axis=0)
y = np.stack(c_out_dat)


**Creating and training the RNN model**

In [0]:
val_idx = get_cv_idxs(len(idx)-cs-1)
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)


In [0]:
class rnnmodel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [0]:
model=rnnmodel(vocab_size,n_fac).cuda()
opt=optim.Adam(model.parameters(),1e-3)

In [0]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = model(*V(xs))

In [73]:
fit(model, md, 1, opt, F.nll_loss)


epoch      trn_loss   val_loss   
    0      1.831867   1.801626  



[array([1.80163])]

In [0]:
set_lrs(opt,1e-4)#learning rate annealing

In [78]:
fit(model, md, 3, opt, F.nll_loss)


epoch      trn_loss   val_loss   
    0      1.691983   1.691339  
 23%|██▎       | 212/939 [00:06<00:23, 30.78it/s, loss=1.69]

    1      1.649533   1.666025  
    2      1.639801   1.642694  



[array([1.64269])]

## Testing the model

In [79]:
get_next('for thos')


'e'

In [80]:
get_next('I love y')


'o'

In [85]:
get_next("sleepin")


'g'

In [0]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

## Those outputs look pretty good
##End.